## Exploration of map reduce result

---

### Import Libraries

In [1]:
# import libraries
import findspark

# Locate the spark installation
findspark.init()

In [2]:
import pandas as pd
import pyspark as ps
from pyspark.sql.functions import col, sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql import SparkSession


### Initialize Spark

In [3]:
# Initialize a SparkContext
spark = SparkSession.builder.appName("data_cleaning").getOrCreate()
spark.stop()
sc = ps.SparkContext(appName="prior_analysis")
# Initialize the Session
spark_session = ps.sql.SparkSession(sc)

23/09/08 11:18:02 WARN Utils: Your hostname, MacBook-Pro-di-Andrea.local resolves to a loopback address: 127.0.0.1; using 192.168.1.148 instead (on interface en0)
23/09/08 11:18:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/08 11:18:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Connect and import data from HDFS directly into a Spark DataFrame

In [4]:
df_join_result = spark_session.read.csv('hdfs://localhost:9900/user/book_reviews/joined_tables.csv', inferSchema=True, sep='\t')
df_join_result.printSchema()
df_join_result.describe().show()
df_join_result.show(5)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)



23/09/08 11:18:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+--------------------+--------------+---------+------------------+----------------+-----------------+----+--------------------+------+-----------------+--------------------+--------------------+--------------------+
|summary|                 _c0|                 _c1|           _c2|      _c3|               _c4|             _c5|              _c6| _c7|                 _c8|   _c9|             _c10|                _c11|                _c12|                _c13|
+-------+--------------------+--------------------+--------------+---------+------------------+----------------+-----------------+----+--------------------+------+-----------------+--------------------+--------------------+--------------------+
|  count|                1712|                1654|          1708|     1603|              1712|            1712|             1712|   0|                1602|  1712|             1712|                1712|                1711|                1712|
|   mean|           

In [6]:
tmp = df_join_result.limit(300).toPandas()
tmp.head(50)

# Title, description, authors, publisher, publishedDate, categories, Price, User_id, profileName, review/helpfulness, review/score, review/time, review/summary, review/text


,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13
0,Growing Up Plain Among Conservative Wilburite ...,Wil Cooper Quaker theologian teacher and ...,Wilmer Albert Cooper,None,1999,Religion,18.00,None,Scott Perry scottperry usermail com,17/17,5.0,943056000,An excellent insider s account of Conservative...,In a rare display of the ability to bridge two...
1,New Country Houses,This richly illustrated volume showcases the b...,The Images Publishing Group,Images Publishing,2009,Architecture,40.15,None,John Matlock Gunny,13/14,5.0,1123977600,Stretching the Limits of Architecture,When I looked at the cover of this book I saw ...
2,New Country Houses,This richly illustrated volume showcases the b...,The Images Publishing Group,Images Publishing,2009,Architecture,40.15,None,K Glendinning architecture student,10/11,5.0,1180396800,A good start on the typology of the country house,Impressions on the title alone the name of th...
3,New Country Houses,This richly illustrated volume showcases the b...,The Images Publishing Group,Images Publishing,2009,Architecture,40.15,None,Clarence Tyner III,0/0,2.0,1298073600,broad range of styles photos need better colo...,Lots of floor plans and cross sections The ph...
4,Odyssey of Innocents,Tania Abu Ali is just thirteen years old when ...,Lydia Edwards,Archway Publishing,2015-04-27,Fiction,17.95,None,Rebecca Brown rebeccasreads,2/2,4.0,1035936000,a simple enriching memoir,This is a true story that reads like an advent...
5,Vehicular Accident Investigation and Reconstru...,Accident investigation reconstruction is more ...,Donald J Van Kirk,CRC Press,2000-09-19,Law,145.51,None,None,2/4,5.0,973209600,Vehicular Accident Investigation and Reconstru...,This book will appeal to the novice as well as...
6,Vehicular Accident Investigation and Reconstru...,Accident investigation reconstruction is more ...,Donald J Van Kirk,CRC Press,2000-09-19,Law,145.51,None,None,5/8,5.0,971222400,Professional User,This is great book for any police officer leg...
7,Vehicular Accident Investigation and Reconstru...,Accident investigation reconstruction is more ...,Donald J Van Kirk,CRC Press,2000-09-19,Law,145.51,None,Johnny L Elliott,7/9,1.0,978825600,Don t Waste your money on this one,Unfortunately this book is little more than a ...
8,Watch And Ward Classic Books on Cassettes Col...,None,None,None,2002,Electronic journals,19.99,None,H Schneider Hermit,4/5,4.0,1285372800,Return on Investment,In later years Henry James rejected his first...
9,Watch And Ward Classic Books on Cassettes Col...,None,None,None,2002,Electronic journals,19.99,None,Giordano Bruno,3/5,5.0,1284422400,New England Regionalist,Watch and Ward was the first novel of Henry J...


In [ ]:
spark_session.stop()